# Step 1: Analyze the Data (Loading & Cleaning)

Loading the data, exploring its structure, and performing necessary preprocessing to make it suitable for a chatbot application.

In [1]:
import spacy
import requests
import json
import logging     
import re
import pandas as pd
import sys
import os
import warnings
warnings.filterwarnings('ignore')
from openai import OpenAI
from bs4 import BeautifulSoup
from PIL import Image
from sentence_transformers import SentenceTransformer, util
# Adding this directory to sys.path
module_path = os.path.abspath(os.path.join('..', 'app'))
if module_path not in sys.path:
    sys.path.append(module_path)
from database import load_data_into_db, get_db_connection
# Disable tokenizers parallelism to avoid the warning
os.environ["TOKENIZERS_PARALLELISM"] = "false"

# Response Formatting

def format_product_response(product):
    # Format the product details into a string
    return f" Price: {product['Price']:.2f}, Product: {product['Title']}, Description: {product['Cleaned_Description'][:123]}..."

# Corrected file path
file_path = '../data/products.csv'
products_df = pd.read_csv(file_path, delimiter=';', usecols=['ID', 'Title', 'Description', 'Vendor', 'Type', 'Tags', 'Price'])

def clean_html(raw_html):
    if pd.isnull(raw_html):
        return ""
    clean_text = BeautifulSoup(raw_html, "html.parser").text
    return " ".join(clean_text.split())

products_df['Cleaned_Description'] = products_df['Description'].apply(clean_html)
products_df = products_df.dropna(subset=['Title'])
products_df = products_df[products_df['Cleaned_Description'].str.strip() != '']
products_df['Tags'] = products_df['Tags'].apply(lambda x: x.split(',') if pd.notnull(x) else [])

# Step 2: Data Preprocessing & Feature Engineering
We need to clean the data, especially the `Description` column, as it contains HTML tags and other non-text elements that may not be useful for our text processing tasks.

In [2]:
# Assuming spaCy is installed and the model is downloaded

nlp = spacy.load("en_core_web_sm")

def preprocess_text(text):
    doc = nlp(text)
    lemmatized = [token.lemma_ for token in doc if not token.is_stop and not token.is_punct and not token.is_space]
    return " ".join(lemmatized)

products_df['Processed_Description'] = products_df['Cleaned_Description'].apply(preprocess_text)
products_df['Processed_Description'].head()

0     delicacy harmony balance pure form smooth grea...
2     product Description Shipping Care gorgeous str...
10    product description Shipping Care Marrying com...
20    completely natural bug spray perfect nighttime...
22    Cold Flu Relief Immune Support Allergy Relief ...
Name: Processed_Description, dtype: object


2. Extracting Features with Named Entity Recognition (NER)
Extracting entities like materials, benefits from descriptions:

In [3]:
def extract_entities(text):
    doc = nlp(text)
    entities = [(ent.text, ent.label_) for ent in doc.ents]
    return entities

products_df['Entities'] = products_df['Cleaned_Description'].apply(extract_entities)
products_df['Entities'].head()

0     [(calendula, GPE), (xylitol, PERSON), (Cruelty...
2     [(90%, PERCENT), (10%, PERCENT), (21"-24, DATE...
10    [(KP, PERSON), (hours, TIME), (One, CARDINAL),...
20    [(mosquito season, DATE), (Organic, NORP), (Ci...
22    [(Cold & Flu Relief, ORG), (Organic Freshly, O...
Name: Entities, dtype: object

In [4]:
# Function to clean tags and convert them to lowercase
def clean_tags(tags):
    return [tag.lower().strip() for tag in tags]

# Clean the Tags and Type columns
products_df['Tags'] = products_df['Tags'].apply(lambda x: clean_tags(x) if isinstance(x, list) else [])
products_df['Type'] = products_df['Type'].str.lower().str.strip()

# Convert Price to numeric type for sorting
products_df['Price'] = pd.to_numeric(products_df['Price'], errors='coerce')

# RUN ONCE - Save the intermediate cleaned data to a new CSV
cleaned_data = '../data/cleaned_products.csv'
products_df.to_csv(cleaned_data, index=False)

# 3. Loading model and data

In [5]:
# Load the SentenceTransformer model
model = SentenceTransformer('all-MiniLM-L6-v2')

In [6]:
# Load the data into the database
load_data_into_db(products_df, model)

# 4. Integrating OpenAI's API for Enhanced Query Processing

To integrate OpenAI's API, ensure you have an API key and have installed the `openai` Python package.

In [7]:
# openai.api_key = 'your-api-key-here'
client = OpenAI(api_key='your-api-key-here')

def process_query_with_gpt4(query):
    prompt = f"Parse the following user query to identify product attributes and price constraints: '{query}'. List attributes and any specific price constraints."
    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo-preview",
            messages=[{"role": "system", "content": "You are a helpful assistant."}, {"role": "user", "content": prompt}]
        )
        return response.choices[0].message.content.strip()
    except Exception as e:
        logging.error(f'Error processing query with GPT-4: {e}')
        return ""


In [8]:
def pgvector_search(query_embedding, top_k=5):
    # Convert the query_embedding to a list if it's a tensor
    if isinstance(query_embedding, torch.Tensor):
        query_embedding = query_embedding.tolist()
        
    # Connect to the database
    conn = get_db_connection()
    
    # Perform the search
    search_results = find_similar_products(conn, query_embedding, top_k)
    
    # Close the database connection
    conn.close()
    
    return search_results

In [9]:
def semantic_search_with_pgvector(query):
    query_embedding = model.encode(query, convert_to_tensor=True)
    
    # Use pgvector to perform the search
    search_results = pgvector_search(query_embedding, top_k=5)
    
    matched_products = [products_df.iloc[hit] for hit in search_results]
    return pd.DataFrame(matched_products)


# Generating Dynamic Responses with GPT-4

Utilizing GPT-4's advanced text generation capabilities to create responses that are contextually relevant, detailed, and tailored to the user's query and the matched products.

In [10]:
def parse_gpt_response(response):
    attributes_pattern = r"\battributes?:\s*([\w\s,]+?)(?:, with a minimum price of|\.)"
    attributes = re.findall(attributes_pattern, response, re.I)
    
    # Flatten attribute list and filter empty values
    attributes = [attr.strip() for attr in ''.join(attributes).split(',') if attr.strip()]
    
    # Adjust regex for min_price to correctly capture numeric values after "minimum price of"
    min_price_pattern = r"minimum price of (\d+)"
    min_price_match = re.search(min_price_pattern, response, re.I)
    min_price = float(min_price_match.group(1)) if min_price_match else None
    
#     # Flatten attribute list and filter empty values
#     attributes = [attr.strip() for sublist in attributes for attr in sublist.split(',') if attr.strip()]
#     min_price = float(min_price.group(1)) if min_price else None

    return attributes, min_price

# Example test
test_response = "Looking for products with the following attributes: natural, organic, with a minimum price of 20."
attributes, min_price = parse_gpt_response(test_response)
print(f"Extracted Attributes: {attributes}")
print(f"Extracted Min Price: {min_price}")

Extracted Attributes: ['natural', 'organic']
Extracted Min Price: 20.0


In [11]:
# 1

def search_products_by_attributes(attributes, min_price=None):
    # Filter products containing all the attributes
    filtered_products = products_df[
        products_df['Tags'].apply(lambda tags: all(attr in tags for attr in attributes)) |
        products_df['Type'].str.contains('|'.join(attributes), case=False, na=False)
    ]
    
    # Further filter by min price if specified
    if min_price is not None:
        filtered_products = filtered_products[filtered_products['Price'] >= min_price]
    
    # Sort by price
    sorted_products = filtered_products.sort_values(by='Price', ascending=True)
    
#     # Manual test for understanding
#     print("Original number of products:", len(products_df))

#     filtered_by_tags = products_df[products_df['Tags'].apply(lambda tags: all(attr in tags for attr in attributes))]
#     print("Filtered by tags:", len(filtered_by_tags))

#     filtered_by_type = products_df[products_df['Type'].str.contains('|'.join(attributes), case=False, na=False)]
#     print("Filtered by type:", len(filtered_by_type))

#     # Assuming products_df['Price'] is already numeric
#     filtered_by_price = filtered_by_tags[filtered_by_tags['Price'] >= min_price]
#     print("Filtered by price:", len(filtered_by_price))

#     # Combine all filters
#     combined_filters = filtered_by_price
#     print("Combined filters:", len(combined_filters))
    
    return sorted_products

In [12]:
# 2 

def filter_and_format_products(products_df, attributes, min_price):
    # Filter products based on attributes and min_price
    filtered_products = products_df[
        (products_df['Price'] >= min_price) & 
        (products_df['Tags'].apply(lambda tags: all(attr in tags for attr in attributes)))
    ]

    # Format the filtered products
    formatted_products = "\n".join([
        f"{i+1}. Price: {row['Price']}, Description: {row['Description'][:100]}..."
        for i, row in filtered_products.iterrows()
    ])
    
    return formatted_products


# # Define test attributes
# attributes_test = ['organic']  # Adjust based on your data
# min_price_test = 20  # Adjust based on your data

# # Test filtering by tags
# filtered_by_tags = products_df[products_df['Tags'].apply(lambda tags: all(attr in tags for attr in attributes_test))]
# print("Filtered by tags:", len(filtered_by_tags))

# # Test filtering by type
# filtered_by_type = products_df[products_df['Type'].str.contains('|'.join(attributes_test), case=False, na=False)]
# print("Filtered by type:", len(filtered_by_type))

# # Test filtering by price
# filtered_by_price = products_df[products_df['Price'] >= min_price_test]
# print("Filtered by price:", len(filtered_by_price))

# # Combine all filters
# combined_filters = filtered_by_tags[filtered_by_tags['Price'] >= min_price_test]
# print("Filtered by tags and price:", len(combined_filters))



In [13]:
def generate_dynamic_response_with_gpt4(query):
    try:
        # Extract attributes and price constraints from the user's query
        processed_response = process_query_with_gpt4(query)
        
        # Parse the processed_response to get attributes and min_price
        attributes, min_price = parse_gpt_response(processed_response)
        
        # Search for products based on extracted attributes and price
        product_results = search_products_by_attributes(attributes, min_price)

        # Format each product and prepend with numbering
        response_strings = [
            f"{i + 1}. {format_product_response(row)}"
            for i, (index, row) in enumerate(product_results.iterrows())
        ]
        
        # Join all formatted product strings into a single response string
        response_string = "\n".join(response_strings)

        return response_string if response_strings else "No products found matching your criteria."
    except requests.exceptions.HTTPError as http_err:
        logging.error(f'HTTP error occurred: {http_err}')
    except Exception as err:
        logging.error(f'Other error occurred: {err}')
    return "Error generating response. Please try again later."


In [14]:
# Sample query from the user
# user_query = "Haircare products"
# user_query = "I want to buy a pair of haircare products with minimal price" # None
user_query = "I'm looking for eco-friendly skincare products with a minimum price of 20"

# Get the response from GPT-4
gpt_response = generate_dynamic_response_with_gpt4(user_query)

# Check and print the response from GPT-4
print(f"GPT-4 Response: \n\n{gpt_response}\n\n")

# Parse the response to extract attributes and min price
attributes, min_price = parse_gpt_response(gpt_response)

# Print the parsed information
# print(f"Extracted Attributes: {attributes}\n")
# print(f"Extracted Min Price: {min_price}\n")

GPT-4 Response: 

1.  Price: 9.68, Product: Rosemary Nettle Shampoo Bar, Description: This rosemary nettle shampoo bar is all you need for thick gorgeous hair!Rosemary is queen and king when it comes to hair a...
2.  Price: 10.94, Product: Handmade Organic Vapor Rub, Description: This Vapor rub is a great alternative to the commercial rubs you can buy in the grocery store for many reasons. First of al...
3.  Price: 11.07, Product: Coconut Silk Conditioning Shampoo Bar, Description: The newest addition to my shampoo soap line - coconut silk conditioning shampoo bar. An incredible bar with coconut milk, s...
4.  Price: 15.29, Product: Shampoo for sensible hairs (SMOOTH), Description: Delicacy, harmony and balance in its purest form - that's Smooth. Great hair is all about balance, but it all starts with a...
5.  Price: 16.55, Product: Organic Bug Spray Bug Repellant, Description: This completely natural bug spray is perfect for those nighttime hikes in mosquito season. Made with a base o